In [9]:
import glob
import random
import pandas as pd
from IPython.display import Image

from functools import partial
from multiprocessing import Pool

import numpy as np

import sklearn
import tsfresh



In [10]:
path = 'A_basins_total_upstrm/2_timeseries/daily'
files = glob.glob(path+'/*')

In [11]:
Image(url="https://zenodo.org/api/iiif/v2/5e7b63a7-7bea-46f6-a8ac-00d63e3cb9b1:c5697b0f-87ce-4b8d-94b2-a8296e3367d1:LamaH_banner.png/full/750,/0/default.png", width=300, height=300)


In [12]:
random_number=1
shuff = True
random.seed(random_number)
if shuff:
    random.shuffle(files)
    files = files[:100]
else:
    files=files[:100]

In [13]:
df_list = (pd.read_csv(file,delimiter=';') for file in files)

In [14]:
# We also need to extract what file they came from for the ID
df = pd.DataFrame()
for i,df2 in enumerate(df_list):
    df2["id"]=int(files[i].split("ID_")[1].strip(".csv"))
    df   = pd.concat([df,df2], ignore_index=True)

In [15]:
#df   = pd.concat(df_list, ignore_index=True)

In [16]:
#column_to_move = df.pop("prec")
#df.insert(len(df.columns), "prec", column_to_move)

In [17]:
# time is a counter per group. Later used to filter out first x Observations for each group
df['time'] = df.groupby('id').cumcount()

# the indipendent variables
var_cols=['2m_temp_max', '2m_temp_mean', '2m_temp_min',
       '2m_dp_temp_max', '2m_dp_temp_mean', '2m_dp_temp_min', '10m_wind_u',
       '10m_wind_v', 'fcst_alb', 'lai_high_veg', 'lai_low_veg', 'swe',
       'surf_net_solar_rad_max', 'surf_net_solar_rad_mean',
       'surf_net_therm_rad_max', 'surf_net_therm_rad_mean', 'surf_press',
       'total_et', 'volsw_123', 'volsw_4',"prec"]

In [18]:
# Target variable is the perc once shifted
df["target"] = df["prec"].shift(periods=-1)

In [19]:

# creating means and lagging columns per inidpendent variable
for col in var_cols:
    
    df[col+"_mean_3d"] = df[col].rolling(3).mean()
    df[col+"_std_3d"] = df[col].rolling(3).std()
    
    df[col+"_mean_7d"] = df[col].rolling(7).mean()
    df[col+"_std_7d"] = df[col].rolling(7).std()
    
    df[col+"_mean_30d"] = df[col].rolling(30).mean()
    df[col+"_std_30d"] = df[col].rolling(30).std()

    df[col+"_mean_365d"] = df[col].rolling(30).mean()
    df[col+"_std_365d"] = df[col].rolling(30).std()

    # Create features using differencing
    df["diff_1d"] = df[col].diff(1)
    df["diff_2d"] = df[col].diff(2)
    df["diff_365d"] = df[col].diff(365)

    # Create features using lagging
    df[col+"_lag_1d"] = df[col].shift(periods=1)
    df[col+"_lag_2d"] = df[col].shift(periods=2)
    df[col+"_lag_365d"] = df[col].shift(periods=365)

    # Create features using window functions
    df[col+"_rolling_mean_14d"] = df[col].rolling(window=14).mean()
    #break
    
#df.head(10)



C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_lag_2d"] = df[col].shift(periods=2)
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_lag_365d"] = df[col].shift(periods=365)
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance

C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_std_365d"] = df[col].rolling(30).std()
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_lag_1d"] = df[col].shift(periods=1)
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_std_365d"] = df[col].rolling(30).std()
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_lag_1d"] = df[col].shift(periods=1)
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_mean_365d"] = df[col].rolling(30).mean()
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_std_365d"] = df[col].rolling(30).std()
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_mean_365d"] = df[col].rolling(30).mean()
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_std_365d"] = df[col].rolling(30).std()
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_std_30d"] = df[col].rolling(30).std()
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_mean_365d"] = df[col].rolling(30).mean()
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_mean_30d"] = df[col].rolling(30).mean()
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+"_std_30d"] = df[col].rolling(30).std()
C:\Users\alexs\AppData\Local\Temp\ipykernel_23948\4286204180.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

In [20]:
# fist 366 have nans since i cant build the average or lag if i dont have the data
df =df.loc[df["time"]>366]


# valid set is from 2016 till 2019 (inclusive)
train = df.loc[df["YYYY"]<2016]
valid = df.loc[df["YYYY"]>=2016]

# we dont know what the actual y for the prediction of the last day is so we need to drop it
valid=valid.loc[~((valid["YYYY"]==2019) & (valid["MM"]==12) & (valid["DD"]==31))]

train=train.drop(["YYYY","MM","DD","DOY","id","time"],axis=1)
valid=valid.drop(["YYYY","MM","DD","DOY","id","time"],axis=1)




X_train = train.drop("target", axis=1)
X_valid = valid.drop("target", axis=1)
y_train = train["target"]
y_valid = valid["target"]


## Ridge

In [ ]:
%%time
from sklearn.linear_model import Ridge
model=Ridge()
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_valid)

from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_valid, y_pred, squared=False)
rms

In [ ]:
importance = model.coef_
# summarize feature importance
for i,v in enumerate(importance):
 print('Feature: %s, Score: %.4f' % (X_train.columns[i],v))

## Random Forest

In [24]:
%%time
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(random_state=random_number,max_depth=10,n_jobs=-1)
model.fit(X_train,y_train)
y_pred=model.predict(X_valid)
rms2 = mean_squared_error(y_valid, y_pred, squared=False)
rms2


KeyboardInterrupt: 